## Matrial_projectからLiイオンを含む化合物データを取得し、分析用データを作成

In [ ]:
# 必要なライブラリのインストール
!pip install -q pymatgen
!pip install -q matminer
!pip install -q  mp_api

In [2]:
from mp_api.client import MPRester
import pandas as pd
import os

## material projectのbattery explorerからリチウムイオンの化合物のデータを取得

In [3]:
#API_KEYの設定
from google.colab import userdata
MY_API_KEY = userdata.get('mp_API_KEY')

mpr = MPRester(MY_API_KEY)

In [4]:
# battery explorerから取得できるデータの一覧
print(mpr.materials.insertion_electrodes.available_fields)

['battery_type', 'battery_id', 'thermo_type', 'battery_formula', 'working_ion', 'num_steps', 'max_voltage_step', 'last_updated', 'framework', 'framework_formula', 'elements', 'nelements', 'chemsys', 'formula_anonymous', 'warnings', 'formula_charge', 'formula_discharge', 'max_delta_volume', 'average_voltage', 'capacity_grav', 'capacity_vol', 'energy_grav', 'energy_vol', 'fracA_charge', 'fracA_discharge', 'stability_charge', 'stability_discharge', 'id_charge', 'id_discharge', 'host_structure', 'adj_pairs', 'material_ids', 'entries_composition_summary', 'electrode_object']


## 取得できるデータの概要
- battery_type : バッテリータイプ
- battery_id : バッテリーID
- thermo_type : 熱力学タイプ
- battery_formula : バッテリーの化学式
- working_ion : 移動するイオン
- num_steps : ステップ数
- max_voltage_step : 最大電圧ステップ
- last_updated : 最終更新日
- framework : 移動するイオンを除いた原子
- framework_formula : 移動するイオンを除いた化学式
- elements : 元素
- nelements : 元素の数
- chemsys : 元素系
- formula_anonymous : 匿名化された化学式
- warnings : 警告
- formula_charge : 充電時の化学式
- formula_discharge : 放電時の化学式
- max_delta_volume : 最大体積変化
- average_voltage : 平均電圧
- capacity_grav : 重量容量
- capacity_vol : 体積容量
- energy_grav : 重量エネルギー
- energy_vol : 体積エネルギー
- fracA_charge : 充電時の成分比率
- fracA_discharge : 放電時の成分比率
- stability_charge : 充電時の安定性
- stability_discharge : 放電時の安定性
- id_charge : 充電時のID
- id_discharge : 放電時のID
- host_structure : ホスト構造
- adj_pairs : 隣接ペア
- material_ids : マテリアルID
- entries_composition_summary : 組成の概要
- electrode_object : 電極オブジェクト


In [5]:
# Material ProjectからLiイオンを含む化合物データを取得
Li_dataset = mpr.materials.insertion_electrodes.search(working_ion="Li")

Retrieving InsertionElectrodeDoc documents:   0%|          | 0/2440 [00:00<?, ?it/s]

In [6]:
# 取得したデータ数の確認
len(Li_dataset)

2440

## 取得したデータからデータフレームを作成

In [7]:
# 取得したデータからデータフレーム用の列名を作成
columns = Li_dataset[0].dict().keys()
columns

dict_keys(['battery_type', 'battery_id', 'thermo_type', 'battery_formula', 'working_ion', 'num_steps', 'max_voltage_step', 'last_updated', 'framework', 'framework_formula', 'elements', 'nelements', 'chemsys', 'formula_anonymous', 'warnings', 'formula_charge', 'formula_discharge', 'max_delta_volume', 'average_voltage', 'capacity_grav', 'capacity_vol', 'energy_grav', 'energy_vol', 'fracA_charge', 'fracA_discharge', 'stability_charge', 'stability_discharge', 'id_charge', 'id_discharge', 'host_structure', 'adj_pairs', 'material_ids', 'entries_composition_summary', 'electrode_object', 'fields_not_requested'])

In [8]:
# データフレームに格納するデータの準備
Li_data = []

for i in range(len(Li_dataset)):
    Li_value = Li_dataset[i].dict().values()
    Li_data.append(Li_value)

In [10]:
# 取得したデータをデータフレームに格納
df = pd.DataFrame(Li_data, columns=columns)
df.head()

,battery_type,battery_id,thermo_type,battery_formula,working_ion,num_steps,max_voltage_step,last_updated,framework,framework_formula,...,stability_charge,stability_discharge,id_charge,id_discharge,host_structure,adj_pairs,material_ids,entries_composition_summary,electrode_object,fields_not_requested
0,None,mp-757871_Li,None,Li2-4CoNi3O8,Li,2,0.221134,2022-06-28 03:50:55.322000,"{'Co': 1.0, 'Ni': 3.0, 'O': 8.0}",CoNi3O8,...,0.037008,0.010849,mp-1178027,mp-867640,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Li2CoNi3O8', 'formula_dis...","[mp-1178027, mp-757871, mp-867640]","{'all_formulas': ['Li4CoNi3O8', 'Li2CoNi3O8', ...",{'voltage_pairs': [{'voltage': 3.7556553933333...,"[battery_type, thermo_type]"
1,None,mp-755220_Li,None,Li3-4CrNi3O8,Li,1,0.000000,2022-06-28 03:50:55.322000,"{'Cr': 1.0, 'Ni': 3.0, 'O': 8.0}",CrNi3O8,...,0.089504,0.038122,mp-755220,mp-757137,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Li3CrNi3O8', 'formula_dis...","[mp-755220, mp-757137]","{'all_formulas': ['Li4CrNi3O8', 'Li3CrNi3O8'],...",{'voltage_pairs': [{'voltage': 3.5559580133333...,"[battery_type, thermo_type]"
2,None,mp-762224_Li,None,Li0-4Cu3Sb(PO4)4,Li,1,0.000000,2022-06-28 03:50:55.322000,"{'Cu': 3.0, 'Sb': 1.0, 'P': 4.0, 'O': 16.0}",Cu3Sb(PO4)4,...,0.067563,0.072026,mp-780043,mp-762224,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Cu3Sb(PO4)4', 'formula_di...","[mp-780043, mp-762224]","{'all_formulas': ['Li4Cu3Sb(PO4)4', 'Cu3Sb(PO4...",{'voltage_pairs': [{'voltage': 3.6375136033333...,"[battery_type, thermo_type]"
3,None,mp-759472_Li,None,Li1-2Cu(HO)4,Li,1,0.000000,2022-06-28 03:50:55.322000,"{'Cu': 1.0, 'H': 4.0, 'O': 4.0}",Cu(HO)4,...,0.078001,0.080676,mp-759472,mp-769242,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'LiCu(HO)4', 'formula_disc...","[mp-759472, mp-769242]","{'all_formulas': ['Li2Cu(HO)4', 'LiCu(HO)4'], ...",{'voltage_pairs': [{'voltage': 2.9016335933333...,"[battery_type, thermo_type]"
4,None,mp-772496_Li,None,Li0-4Mn3CuNi2(PO4)6,Li,1,0.000000,2022-06-28 03:50:55.322000,"{'Mn': 3.0, 'Cu': 1.0, 'Ni': 2.0, 'P': 6.0, 'O...",Mn3CuNi2(PO4)6,...,0.045275,0.082965,mp-772496,mp-775323,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Mn3CuNi2(PO4)6', 'formula...","[mp-772496, mp-775323]","{'all_formulas': ['Mn3CuNi2(PO4)6', 'Li4Mn3CuN...",{'voltage_pairs': [{'voltage': 3.7576910933333...,"[battery_type, thermo_type]"


In [11]:
# データ確認
df.shape

(2440, 35)

In [12]:
# データ情報
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2440 entries, 0 to 2439
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   battery_type                 0 non-null      object 
 1   battery_id                   2440 non-null   object 
 2   thermo_type                  0 non-null      object 
 3   battery_formula              2440 non-null   object 
 4   working_ion                  2440 non-null   object 
 5   num_steps                    2440 non-null   int64  
 6   max_voltage_step             2440 non-null   float64
 7   last_updated                 2440 non-null   object 
 8   framework                    2440 non-null   object 
 9   framework_formula            2440 non-null   object 
 10  elements                     2440 non-null   object 
 11  nelements                    2440 non-null   int64  
 12  chemsys                      2440 non-null   object 
 13  formula_anonymous 

## データの整理

In [13]:
# id情報の確認
df[["battery_id", "id_charge", "id_discharge", "material_ids"]]

,battery_id,id_charge,id_discharge,material_ids
0,mp-757871_Li,mp-1178027,mp-867640,"[mp-1178027, mp-757871, mp-867640]"
1,mp-755220_Li,mp-755220,mp-757137,"[mp-755220, mp-757137]"
2,mp-762224_Li,mp-780043,mp-762224,"[mp-780043, mp-762224]"
3,mp-759472_Li,mp-759472,mp-769242,"[mp-759472, mp-769242]"
4,mp-772496_Li,mp-772496,mp-775323,"[mp-772496, mp-775323]"
...,...,...,...,...
2435,mp-1235320_Li,mp-1446045,mp-1235320,"[mp-1446045, mp-1235320, mp-1393040]"
2436,mp-760992_Li,mp-764115,mp-772164,"[mp-764115, mp-772164, mp-760992]"
2437,mp-19408_Li,mp-25288,mp-19408,"[mp-25288, mp-19408]"
2438,mp-25207_Li,mp-25207,mp-27751,"[mp-25207, mp-776037, mp-27751, mp-1235037, mp..."


In [14]:
# 不要な変数の削除
clean_df = df.drop(["warnings", "last_updated", "working_ion"], axis=1)
clean_df.head()

,battery_type,battery_id,thermo_type,battery_formula,num_steps,max_voltage_step,framework,framework_formula,elements,nelements,...,stability_charge,stability_discharge,id_charge,id_discharge,host_structure,adj_pairs,material_ids,entries_composition_summary,electrode_object,fields_not_requested
0,None,mp-757871_Li,None,Li2-4CoNi3O8,2,0.221134,"{'Co': 1.0, 'Ni': 3.0, 'O': 8.0}",CoNi3O8,"[Co, Ni, O]",3,...,0.037008,0.010849,mp-1178027,mp-867640,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Li2CoNi3O8', 'formula_dis...","[mp-1178027, mp-757871, mp-867640]","{'all_formulas': ['Li4CoNi3O8', 'Li2CoNi3O8', ...",{'voltage_pairs': [{'voltage': 3.7556553933333...,"[battery_type, thermo_type]"
1,None,mp-755220_Li,None,Li3-4CrNi3O8,1,0.000000,"{'Cr': 1.0, 'Ni': 3.0, 'O': 8.0}",CrNi3O8,"[Cr, Ni, O]",3,...,0.089504,0.038122,mp-755220,mp-757137,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Li3CrNi3O8', 'formula_dis...","[mp-755220, mp-757137]","{'all_formulas': ['Li4CrNi3O8', 'Li3CrNi3O8'],...",{'voltage_pairs': [{'voltage': 3.5559580133333...,"[battery_type, thermo_type]"
2,None,mp-762224_Li,None,Li0-4Cu3Sb(PO4)4,1,0.000000,"{'Cu': 3.0, 'Sb': 1.0, 'P': 4.0, 'O': 16.0}",Cu3Sb(PO4)4,"[Cu, Sb, P, O]",4,...,0.067563,0.072026,mp-780043,mp-762224,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Cu3Sb(PO4)4', 'formula_di...","[mp-780043, mp-762224]","{'all_formulas': ['Li4Cu3Sb(PO4)4', 'Cu3Sb(PO4...",{'voltage_pairs': [{'voltage': 3.6375136033333...,"[battery_type, thermo_type]"
3,None,mp-759472_Li,None,Li1-2Cu(HO)4,1,0.000000,"{'Cu': 1.0, 'H': 4.0, 'O': 4.0}",Cu(HO)4,"[Cu, H, O]",3,...,0.078001,0.080676,mp-759472,mp-769242,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'LiCu(HO)4', 'formula_disc...","[mp-759472, mp-769242]","{'all_formulas': ['Li2Cu(HO)4', 'LiCu(HO)4'], ...",{'voltage_pairs': [{'voltage': 2.9016335933333...,"[battery_type, thermo_type]"
4,None,mp-772496_Li,None,Li0-4Mn3CuNi2(PO4)6,1,0.000000,"{'Mn': 3.0, 'Cu': 1.0, 'Ni': 2.0, 'P': 6.0, 'O...",Mn3CuNi2(PO4)6,"[Mn, Cu, Ni, P, O]",5,...,0.045275,0.082965,mp-772496,mp-775323,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Mn3CuNi2(PO4)6', 'formula...","[mp-772496, mp-775323]","{'all_formulas': ['Mn3CuNi2(PO4)6', 'Li4Mn3CuN...",{'voltage_pairs': [{'voltage': 3.7576910933333...,"[battery_type, thermo_type]"


In [16]:
# 欠損値確認
clean_df.isnull().sum()

battery_type                   2440
battery_id                        0
thermo_type                    2440
battery_formula                   0
num_steps                         0
max_voltage_step                  0
framework                         0
framework_formula                 0
elements                          0
nelements                         0
chemsys                           0
formula_anonymous                 0
formula_charge                    0
formula_discharge                 0
max_delta_volume                  0
average_voltage                   0
capacity_grav                     0
capacity_vol                      0
energy_grav                       0
energy_vol                        0
fracA_charge                      0
fracA_discharge                   0
stability_charge                  0
stability_discharge               0
id_charge                         0
id_discharge                      0
host_structure                    0
adj_pairs                   

In [17]:
# 欠損値のある列を削除
clean_df = clean_df.dropna(axis=1)
clean_df.head()

,battery_id,battery_formula,num_steps,max_voltage_step,framework,framework_formula,elements,nelements,chemsys,formula_anonymous,...,stability_charge,stability_discharge,id_charge,id_discharge,host_structure,adj_pairs,material_ids,entries_composition_summary,electrode_object,fields_not_requested
0,mp-757871_Li,Li2-4CoNi3O8,2,0.221134,"{'Co': 1.0, 'Ni': 3.0, 'O': 8.0}",CoNi3O8,"[Co, Ni, O]",3,Co-Ni-O,AB3C8,...,0.037008,0.010849,mp-1178027,mp-867640,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Li2CoNi3O8', 'formula_dis...","[mp-1178027, mp-757871, mp-867640]","{'all_formulas': ['Li4CoNi3O8', 'Li2CoNi3O8', ...",{'voltage_pairs': [{'voltage': 3.7556553933333...,"[battery_type, thermo_type]"
1,mp-755220_Li,Li3-4CrNi3O8,1,0.000000,"{'Cr': 1.0, 'Ni': 3.0, 'O': 8.0}",CrNi3O8,"[Cr, Ni, O]",3,Cr-Ni-O,AB3C8,...,0.089504,0.038122,mp-755220,mp-757137,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Li3CrNi3O8', 'formula_dis...","[mp-755220, mp-757137]","{'all_formulas': ['Li4CrNi3O8', 'Li3CrNi3O8'],...",{'voltage_pairs': [{'voltage': 3.5559580133333...,"[battery_type, thermo_type]"
2,mp-762224_Li,Li0-4Cu3Sb(PO4)4,1,0.000000,"{'Cu': 3.0, 'Sb': 1.0, 'P': 4.0, 'O': 16.0}",Cu3Sb(PO4)4,"[Cu, Sb, P, O]",4,Cu-O-P-Sb,AB3C4D16,...,0.067563,0.072026,mp-780043,mp-762224,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Cu3Sb(PO4)4', 'formula_di...","[mp-780043, mp-762224]","{'all_formulas': ['Li4Cu3Sb(PO4)4', 'Cu3Sb(PO4...",{'voltage_pairs': [{'voltage': 3.6375136033333...,"[battery_type, thermo_type]"
3,mp-759472_Li,Li1-2Cu(HO)4,1,0.000000,"{'Cu': 1.0, 'H': 4.0, 'O': 4.0}",Cu(HO)4,"[Cu, H, O]",3,Cu-H-O,AB4C4,...,0.078001,0.080676,mp-759472,mp-769242,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'LiCu(HO)4', 'formula_disc...","[mp-759472, mp-769242]","{'all_formulas': ['Li2Cu(HO)4', 'LiCu(HO)4'], ...",{'voltage_pairs': [{'voltage': 2.9016335933333...,"[battery_type, thermo_type]"
4,mp-772496_Li,Li0-4Mn3CuNi2(PO4)6,1,0.000000,"{'Mn': 3.0, 'Cu': 1.0, 'Ni': 2.0, 'P': 6.0, 'O...",Mn3CuNi2(PO4)6,"[Mn, Cu, Ni, P, O]",5,Cu-Mn-Ni-O-P,AB2C3D6E24,...,0.045275,0.082965,mp-772496,mp-775323,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Mn3CuNi2(PO4)6', 'formula...","[mp-772496, mp-775323]","{'all_formulas': ['Mn3CuNi2(PO4)6', 'Li4Mn3CuN...",{'voltage_pairs': [{'voltage': 3.7576910933333...,"[battery_type, thermo_type]"


In [18]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2440 entries, 0 to 2439
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   battery_id                   2440 non-null   object 
 1   battery_formula              2440 non-null   object 
 2   num_steps                    2440 non-null   int64  
 3   max_voltage_step             2440 non-null   float64
 4   framework                    2440 non-null   object 
 5   framework_formula            2440 non-null   object 
 6   elements                     2440 non-null   object 
 7   nelements                    2440 non-null   int64  
 8   chemsys                      2440 non-null   object 
 9   formula_anonymous            2440 non-null   object 
 10  formula_charge               2440 non-null   object 
 11  formula_discharge            2440 non-null   object 
 12  max_delta_volume             2440 non-null   float64
 13  average_voltage   

In [19]:
clean_df.describe()

,num_steps,max_voltage_step,nelements,max_delta_volume,average_voltage,capacity_grav,capacity_vol,energy_grav,energy_vol,fracA_charge,fracA_discharge,stability_charge,stability_discharge
count,2440.000000,2440.000000,2440.000000,2440.000000,2440.000000,2440.000000,2440.000000,2440.000000,2440.000000,2440.000000,2440.000000,2440.000000,2440.000000
mean,1.235246,0.219808,3.056557,0.050746,3.571901,137.959898,480.416149,492.954097,1690.615874,0.050828,0.165106,0.135542,0.111799
std,0.541424,0.795967,0.643571,0.124020,1.882350,77.183964,271.069668,346.982963,1155.509423,0.087123,0.089544,0.470353,0.413256
min,1.000000,0.000000,1.000000,0.000018,-7.754751,6.705499,46.344695,-583.545844,-1721.118698,0.000000,0.014286,0.000000,0.000000
25%,1.000000,0.000000,3.000000,0.013692,2.942477,85.205564,287.067122,283.376403,958.471624,0.000000,0.100000,0.032168,0.022448
50%,1.000000,0.000000,3.000000,0.033575,3.614255,124.754600,426.279510,450.043711,1546.565114,0.000000,0.142857,0.065093,0.043807
75%,1.000000,0.000000,3.000000,0.065285,4.244416,171.513967,598.265700,639.914505,2216.367318,0.090909,0.222222,0.093498,0.075038
max,6.000000,26.960688,5.000000,5.158052,54.568825,1104.166948,2085.178415,5926.949724,18305.897652,0.545455,0.750000,6.487098,6.277809


In [20]:
# object型のデータを確認
object_data = clean_df.select_dtypes(include='object')
object_data

,battery_id,battery_formula,framework,framework_formula,elements,chemsys,formula_anonymous,formula_charge,formula_discharge,id_charge,id_discharge,host_structure,adj_pairs,material_ids,entries_composition_summary,electrode_object,fields_not_requested
0,mp-757871_Li,Li2-4CoNi3O8,"{'Co': 1.0, 'Ni': 3.0, 'O': 8.0}",CoNi3O8,"[Co, Ni, O]",Co-Ni-O,AB3C8,Li2CoNi3O8,Li4CoNi3O8,mp-1178027,mp-867640,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Li2CoNi3O8', 'formula_dis...","[mp-1178027, mp-757871, mp-867640]","{'all_formulas': ['Li4CoNi3O8', 'Li2CoNi3O8', ...",{'voltage_pairs': [{'voltage': 3.7556553933333...,"[battery_type, thermo_type]"
1,mp-755220_Li,Li3-4CrNi3O8,"{'Cr': 1.0, 'Ni': 3.0, 'O': 8.0}",CrNi3O8,"[Cr, Ni, O]",Cr-Ni-O,AB3C8,Li3CrNi3O8,Li4CrNi3O8,mp-755220,mp-757137,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Li3CrNi3O8', 'formula_dis...","[mp-755220, mp-757137]","{'all_formulas': ['Li4CrNi3O8', 'Li3CrNi3O8'],...",{'voltage_pairs': [{'voltage': 3.5559580133333...,"[battery_type, thermo_type]"
2,mp-762224_Li,Li0-4Cu3Sb(PO4)4,"{'Cu': 3.0, 'Sb': 1.0, 'P': 4.0, 'O': 16.0}",Cu3Sb(PO4)4,"[Cu, Sb, P, O]",Cu-O-P-Sb,AB3C4D16,Cu3Sb(PO4)4,Li4Cu3Sb(PO4)4,mp-780043,mp-762224,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Cu3Sb(PO4)4', 'formula_di...","[mp-780043, mp-762224]","{'all_formulas': ['Li4Cu3Sb(PO4)4', 'Cu3Sb(PO4...",{'voltage_pairs': [{'voltage': 3.6375136033333...,"[battery_type, thermo_type]"
3,mp-759472_Li,Li1-2Cu(HO)4,"{'Cu': 1.0, 'H': 4.0, 'O': 4.0}",Cu(HO)4,"[Cu, H, O]",Cu-H-O,AB4C4,LiCu(HO)4,Li2Cu(HO)4,mp-759472,mp-769242,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'LiCu(HO)4', 'formula_disc...","[mp-759472, mp-769242]","{'all_formulas': ['Li2Cu(HO)4', 'LiCu(HO)4'], ...",{'voltage_pairs': [{'voltage': 2.9016335933333...,"[battery_type, thermo_type]"
4,mp-772496_Li,Li0-4Mn3CuNi2(PO4)6,"{'Mn': 3.0, 'Cu': 1.0, 'Ni': 2.0, 'P': 6.0, 'O...",Mn3CuNi2(PO4)6,"[Mn, Cu, Ni, P, O]",Cu-Mn-Ni-O-P,AB2C3D6E24,Mn3CuNi2(PO4)6,Li4Mn3CuNi2(PO4)6,mp-772496,mp-775323,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Mn3CuNi2(PO4)6', 'formula...","[mp-772496, mp-775323]","{'all_formulas': ['Mn3CuNi2(PO4)6', 'Li4Mn3CuN...",{'voltage_pairs': [{'voltage': 3.7576910933333...,"[battery_type, thermo_type]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,mp-1235320_Li,Li0-0.12VO2,"{'V': 1.0, 'O': 2.0}",VO2,"[V, O]",O-V,AB2,VO2,LiV8O16,mp-1446045,mp-1235320,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'VO2', 'formula_discharge'...","[mp-1446045, mp-1235320, mp-1393040]","{'all_formulas': ['VO2', 'LiV8O16'], 'all_chem...",{'voltage_pairs': [{'voltage': 2.7833067433333...,"[battery_type, thermo_type]"
2436,mp-760992_Li,Li0-2V3O7,"{'V': 3.0, 'O': 7.0}",V3O7,"[V, O]",O-V,A3B7,V3O7,Li2V3O7,mp-764115,mp-772164,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'V3O7', 'formula_discharge...","[mp-764115, mp-772164, mp-760992]","{'all_formulas': ['Li2V3O7', 'V3O7'], 'all_che...",{'voltage_pairs': [{'voltage': 3.2735610533333...,"[battery_type, thermo_type]"
2437,mp-19408_Li,Li0-1V2O5,"{'V': 2.0, 'O': 5.0}",V2O5,"[V, O]",O-V,A2B5,V2O5,LiV2O5,mp-25288,mp-19408,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'V2O5', 'formula_discharge...","[mp-25288, mp-19408]","{'all_formulas': ['V2O5', 'LiV2O5'], 'all_chem...",{'voltage_pairs': [{'voltage': 3.4979915308333...,"[battery_type, thermo_type]"
2438,mp-25207_Li,Li0-0.67V2O5,"{'V': 2.0, 'O': 5.0}",V2O5,"[V, O]",O-V,A2B5,V2O5,Li2V6O15,mp-25207,mp-27751,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'V2O5', 'formula_discharge...","[mp-25207, mp-776037, mp-27751, mp-1235037, mp...","{'all_formulas': ['Li2V6O15', 'V2O5', 'LiV4O10...",{'voltage_pairs': [{'voltage': 3.6917310599999...,"[battery_type, thermo_type]"


In [21]:
# 数値型のデータを確認
float_data = clean_df.select_dtypes(include='float64')
float_data

,max_voltage_step,max_delta_volume,average_voltage,capacity_grav,capacity_vol,energy_grav,energy_vol,fracA_charge,fracA_discharge,stability_charge,stability_discharge
0,0.221134,0.008674,3.645088,137.171754,644.437651,500.003153,2349.032129,0.142857,0.250000,0.037008,0.010849
1,0.000000,0.003674,3.555958,69.825436,316.717528,248.296319,1126.234230,0.200000,0.250000,0.089504,0.038122
2,0.000000,0.078912,3.637514,148.887306,572.668449,541.579600,2083.089272,0.000000,0.142857,0.067563,0.072026
3,0.000000,0.024693,2.901634,184.256618,518.049443,534.645191,1503.189668,0.100000,0.181818,0.078001,0.080676
4,0.000000,0.071289,3.757691,113.645161,386.424999,427.043409,1452.065776,0.000000,0.100000,0.045275,0.082965
...,...,...,...,...,...,...,...,...,...,...,...
2435,0.000000,0.022063,2.783307,39.974563,137.277501,111.261470,382.085393,0.000000,0.040000,0.137107,0.130874
2436,0.000000,0.091835,3.273561,192.330534,577.524815,629.605747,1890.562742,0.000000,0.166667,0.082159,0.067151
2437,0.000000,0.052291,3.497992,141.941210,444.443066,496.509149,1554.658080,0.000000,0.125000,0.012942,0.025422
2438,0.641710,0.015606,3.531304,95.801349,315.544816,338.303650,1114.284551,0.000000,0.086957,0.029818,0.034136


In [ ]:
# データ保存
clean_df.to_csv("clean_Li_battery_datatest.csv", index=False)